In [30]:
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from utils import AudioSVMClassifier
import os
# import torch

In [31]:
base_directory = "/data/Deep_Fake_Data/"
feat_directory = "Features_no_padding"
data_names = 'CODEC2'
data_types = "train"
# features_ls  = ['ssast_patch_base', 'wavlm_large', 'data2vec_large_ll60k', 'wavlablm_mk_40k', 'unispeech_sat_large', 'hubert_large_ll60k', 'xlsr_53', 'passt_base', 'espnet_hubert_base_iter0', 'mae_ast_patch', 'ast', 'byol_a_2048', 'modified_cpc', 
#                  'byol_s_cvt', 'wav2vec2_base_s2st_es_voxpopuli']
features_ls  =['ssast_patch_base', 'wavlm_large', 'data2vec_large_ll60k', 'wavlablm_mk_40k', 'unispeech_sat_large', 'hubert_large_ll60k', 'xlsr_53', 'passt_base', 'espnet_hubert_base_iter0', 'mae_ast_patch', 'ast', 'byol_a_2048', 'modified_cpc', 
                 'byol_s_cvt', 'wav2vec2_base_s2st_es_voxpopuli']
features_ls = ['byol_a_2048', 'byol_s_cvt',]
speakers_ls = ['p226', 'p227', 'p229', 'p232', 'p236', 'p240', 'p241', 'p243', 'p244', 'p247', 'p250', 'p251', 'p255', 'p258', 'p259', 'p261', 'p262', 'p266', 'p267', 'p268', 'p270', 'p271', 'p272', 'p276', 'p278', 'p283', 'p284', 'p285', 'p287', 'p297', 'p298', 'p304', 'p305', 'p306', 'p311', 'p314', 'p323', 'p326', 'p333', 'p340', 'p341', 'p347', 'p376']
speakers_ls = ['p226']
deepfake_types = ['f01', 'f02', 'f03', 'f04', 'f05', 'f06', 'f07', 'l1', 'l2']
deepfake_types = ['f01']
output_dir = 'one_class_svm'

In [32]:
nu_values = [0.001, 0.01, 0.05, 0.1, 0.2, 0.3]
gamma_values = [0.001, 0.01, 0.1, 1, 10]
kernel_inputs = ['rbf', 'sigmoid', 'poly', 'linear']


In [ ]:
all_results = {}
for speaker in speakers_ls:
    all_results[speaker] = {}
    for feature in features_ls:
        if feature not in all_results[speaker]:
            all_results[speaker][feature] = {}

        for deepfake in deepfake_types:
            current_accuracy = 0
            current_param = None

            for nu in nu_values:
                for kernel in kernel_inputs:
                    for gamma in gamma_values:
                        
                        path_to_original_data = os.path.join(base_directory, feat_directory, data_names, speaker, data_types, 'Original', feature)
                        classifier = AudioSVMClassifier(speaker, base_directory, feature, output_dir)
                        classifier.train(path_to_original_data, nu, gamma, kernel,'OneClassSVM')

                        path_to_deepfake_data = os.path.join(base_directory, feat_directory, data_names, speaker, data_types, deepfake, feature)
                        y_orig, y_pred = classifier.test(path_to_deepfake_data, include_original=True)
                        results = classifier.evaluate_model(compute_accuracy=True, compute_auc=False, compute_conf_matrix=False)
                        accuracy = float(results['accuracy'])

                        if accuracy > current_accuracy:
                            current_accuracy = accuracy
                            current_param = (nu, gamma)

            all_results[speaker][feature][deepfake] = {
                'accuracy': current_accuracy,
                'param': current_param
            }
